# Advent of Code, 5th Dec 2024

## Print Queue - Part 1

Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you, because they waste no time explaining that the new **sleigh launch safety manual** updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a v**ery specific order**. The notation X|Y means that if both page number X and page number Y are to be produced as part of an update, page number X **must** be printed at some point before page number Y.

The Elf has for you both the **page ordering rules** and the **pages to produce in each update** (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

```
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13
```

```
75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
```

The first section specifies the **page ordering rules**, one per line. The first rule, 47|53, means that if an update includes both page number 47 and page number 53, then page number 47 **must** be printed at some point before page number 53. (47 doesn't necessarily need to be **immediately** before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each **update**. Because most safety manuals are different, the pages needed in the updates are different too. The first update, 75,47,61,53,29, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying **which updates are already in the right order**.

In the above example, the first update (75,47,61,53,29) is in the right order:

* 75 is correctly first because there are rules that put each other page after it: 75|47, 75|61, 75|53, and 75|29.
* 47 is correctly second because 75 must be before it (75|47) and every other page must be after it according to 47|61, 47|53, and 47|29.
* 61 is correctly in the middle because 75 and 47 are before it (75|61 and 47|61) and 53 and 29 are after it (61|53 and 61|29).
* 53 is correctly fourth because it is before page number 29 (53|29).
* 29 is the only page left and so is correctly last.

Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, 75,97,47,61,53, is **not** in the correct order: it would print 75 before 97, which violates the rule 97|75.

The fifth update, 61,13,29, is also **not** in the correct order, since it breaks the rule 29|13.

The last update, 97,13,75,29,47, is **not** in the correct order due to breaking several rules.

For some reason, the Elves also need to know the **middle page number** of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

75,47,**61**,53,29

97,61,**53**,29,13

75,**29**,13

These have middle page numbers of 61, 53, and 29 respectively. Adding these page numbers together gives **143**.

Of course, you'll need to be careful: the actual list of **page ordering rules** is bigger and more complicated than the above example.

Determine which updates are already in the correct order. **What do you get if you add up the middle page number from those correctly-ordered updates?**

### Read in the data

In [1]:
#!F#
let inputFilePath = "aoc-2024-12-05-puzzle-input.txt"

Read the lines from the file.

In [2]:
open System.IO

let lines: string list = File.ReadAllLines(inputFilePath) |> Array.toList
lines.Length

1379

### Solve the problem

We need to split the lines into two parts - the page order rules, and then the updates.  They are separated by a blank line.

In [3]:
let pageOrderLines = lines |> List.takeWhile (fun line -> line.Length > 0)
pageOrderLines[0]

53|62

In [4]:
pageOrderLines[pageOrderLines.Length-1]

14|58

In [5]:
let updateLines = lines |> List.skipWhile (fun line -> line.Length > 0) |> List.skip 1
updateLines[0]

71,31,35,38,58,83,43,67,11,45,32

In [6]:
updateLines[updateLines.Length-1]

46,45,48,32,86,67,55,51,59

Now we can check which updates obey all of the page order rules.  First we build a map of which pages must come after a given page, based on the page order rules.

In [7]:
let foldPageOrderMap (map: Map<int,int Set>) (line: string): Map<int,int Set> =
    let keyValuePair = line.Split('|')
    let key = keyValuePair[0] |> Int32.Parse
    let value = keyValuePair[1] |> Int32.Parse
    if (map |> Map.containsKey key)
    then
        let updatedSet: int Set = map[key] |> Set.add value
        map |> Map.add key updatedSet
    else
        let newSet: int Set = [value] |> Set.ofList
        map |> Map.add key newSet

let pageOrderMap = pageOrderLines |> List.fold foldPageOrderMap Map.empty<int,int Set>
pageOrderMap.Count

49

Next we convert the update lines into lists.

In [8]:
let updates = updateLines |> List.map (fun line -> line.Split(',') |> Array.toList |> List.map (fun str -> str |> Int32.Parse))
updates.Length

202

Now we can check which updates page the correct page order.

In [60]:
let checkUpdate (update: int list): bool =
    let rec iterate (update: int list) (previous: int Set): bool =
        match update with
        | [] -> true
        | head::[] -> true
        | head::tail ->
            if ((pageOrderMap |> Map.containsKey head) && not (pageOrderMap[head] |> Set.intersect previous).IsEmpty)
            then
                false // found out of order pages
            else iterate tail (previous |> Set.add head)
    iterate update Set.empty<int>

In [61]:
let correctUpdates, incorrectUpdates = updates |> List.partition checkUpdate
(correctUpdates.Length, incorrectUpdates.Length)

Item1,120
Item2,82


Now we need to sum the middle numbers of each correct update.

In [62]:
correctUpdates |> List.map (fun update -> update.Length) |> Set.ofList

Count,10
(values),"[ 5, 7, 9, 11, 13, 15, 17, 19, 21, 23 ]"


We see that all of the correct updates lists have an odd length, meaning that there is always a well-defined middle value.

In [63]:
let middle (update: int list) =
    update[(update.Length-1)/2]

In [64]:
middle [1;2;3]

2

In [65]:
middle [1;2;3;4;5]

3

In [66]:
correctUpdates |> List.map middle |> List.sum

7307

## Print Queue - Part 2

While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the **incorrectly-ordered updates**, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

* 75,97,47,61,53 becomes 97,75,**47**,61,53.
* 61,13,29 becomes 61,**29**,13.
* 97,13,75,29,47 becomes 97,75,**47**,29,13.

After taking **only the incorrectly-ordered updates** and ordering them correctly, their middle page numbers are 47, 29, and 47. Adding these together produces **123**.

Find the updates which are not in the correct order. **What do you get if you add up the middle page numbers after correctly ordering just those updates?**

In [67]:
let reversePageOrderPairSeq = seq {
    for key in pageOrderMap.Keys do
        let valueSet = pageOrderMap[key]
        for item in valueSet do
            yield (item,key)
}

let foldReversePageOrderMap (map: Map<int,int Set>) (pair: int*int): Map<int,int Set> =
    let key, value = pair
    if (map |> Map.containsKey key)
    then map |> Map.add key (map[key] |> Set.add value)
    else map |> Map.add key ([value] |> Set.ofList)

let reversePageOrderMap = reversePageOrderPairSeq |> Seq.toList |> List.fold foldReversePageOrderMap Map.empty<int,int Set>
reversePageOrderMap.Count

49

In [68]:
let fixOrder (update: int list) =
    let rec iterate (update: int list) (ordered: int list): int list =
        match update with
        | [] -> ordered
        | head::tail ->
            if (reversePageOrderMap |> Map.containsKey head)
            then
                let outOfOrderSet = reversePageOrderMap[head] |> Set.intersect (tail |> Set.ofList)
                let outOfOrderList, inOrderList = update |> List.partition (fun item -> outOfOrderSet.Contains(item))
                if (outOfOrderList.IsEmpty)
                then ordered @ [head] @ (iterate tail List.empty<int>)
                else iterate (ordered @ outOfOrderList @ inOrderList) List.empty<int>
            else
                iterate tail (ordered @ [head])
    let result = iterate update List.empty<int>
    if (not (checkUpdate result))
    then printfn "!!!! fixOrder: failed: %A -> %A" update result
    result

In [70]:
incorrectUpdates[0]

[ 32, 99, 27, 62, 48, 64, 46, 98, 14 ] HeadOrDefault 32 TailOrNull [ 99, 27, 62, 48, 64, 46, 98, 14 ] HeadOrDefault 99 TailOrNull [ 27, 62, 48, 64, 46, 98, 14 ] HeadOrDefault 27 TailOrNull [ 62, 48, 64, 46, 98, 14 ] HeadOrDefault 62 TailOrNull [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 14 ] Head 62 Tail [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 14 ] (values) [ 62, 48, 64, 46, 98, 14 ] Head 27 Tail [ 62, 48, 64, 46, 98, 14 ] HeadOrDefault 62 TailOrNull [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 14 ] Head 62 Tail [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 14 ] (values) [ 62, 48, 64, 46, 98, 14 ] (values) [ 27, 62, 48, 64, 46, 98, 14 ] Head 99 Tail [ 27, 62, 48, 64, 46, 98, 14 ] HeadOrDefault 27 TailOrNull [ 62, 48, 64, 46, 98, 14 ] HeadOrDefault 62 TailOrNull [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 14 ] Head 62 Tail [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 14 ] (values) [ 62, 48, 64, 46, 98, 14 ] Head 27 Tail [ 62, 48, 64, 46, 98, 14 ] HeadOrDefault 62 TailOrNull [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 14 ] Head 62 Tail [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 14 ] (values) [ 62, 48, 64, 46, 98, 14 ] (values) [ 27, 62, 48, 64, 46, 98, 14 ] (values) [ 99, 27, 62, 48, 64, 46, 98, 14 ] Head 32 Tail [ 99, 27, 62, 48, 64, 46, 98, 14 ] HeadOrDefault 99 TailOrNull [ 27, 62, 48, 64, 46, 98, 14 ] HeadOrDefault 27 TailOrNull [ 62, 48, 64, 46, 98, 14 ] HeadOrDefault 62 TailOrNull [ 48, 64, 46, 98, 14 ] HeadOrDefault 48 TailOrNull [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] Head 48 Tail [ 64, 46, 98, 14 ] HeadOrDefault 64 TailOrNull [ 46, 98, 14 ] Head 64 Tail [ 46, 98, 14 ] (values) [ 64, 46, 98, 14 ] (values) [ 48, 64, 46, 98, 

In [72]:
let fixedUpdates = incorrectUpdates |> List.map fixOrder
fixedUpdates[0]

[ 46, 27, 64, 32, 48, 98, 62, 99, 14 ] HeadOrDefault 46 TailOrNull [ 27, 64, 32, 48, 98, 62, 99, 14 ] HeadOrDefault 27 TailOrNull [ 64, 32, 48, 98, 62, 99, 14 ] HeadOrDefault 64 TailOrNull [ 32, 48, 98, 62, 99, 14 ] HeadOrDefault 32 TailOrNull [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 14 ] Head 32 Tail [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 14 ] (values) [ 32, 48, 98, 62, 99, 14 ] Head 64 Tail [ 32, 48, 98, 62, 99, 14 ] HeadOrDefault 32 TailOrNull [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 14 ] Head 32 Tail [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 14 ] (values) [ 32, 48, 98, 62, 99, 14 ] (values) [ 64, 32, 48, 98, 62, 99, 14 ] Head 27 Tail [ 64, 32, 48, 98, 62, 99, 14 ] HeadOrDefault 64 TailOrNull [ 32, 48, 98, 62, 99, 14 ] HeadOrDefault 32 TailOrNull [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 14 ] Head 32 Tail [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 14 ] (values) [ 32, 48, 98, 62, 99, 14 ] Head 64 Tail [ 32, 48, 98, 62, 99, 14 ] HeadOrDefault 32 TailOrNull [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 14 ] Head 32 Tail [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 14 ] (values) [ 32, 48, 98, 62, 99, 14 ] (values) [ 64, 32, 48, 98, 62, 99, 14 ] (values) [ 27, 64, 32, 48, 98, 62, 99, 14 ] Head 46 Tail [ 27, 64, 32, 48, 98, 62, 99, 14 ] HeadOrDefault 27 TailOrNull [ 64, 32, 48, 98, 62, 99, 14 ] HeadOrDefault 64 TailOrNull [ 32, 48, 98, 62, 99, 14 ] HeadOrDefault 32 TailOrNull [ 48, 98, 62, 99, 14 ] HeadOrDefault 48 TailOrNull [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] Head 48 Tail [ 98, 62, 99, 14 ] HeadOrDefault 98 TailOrNull [ 62, 99, 14 ] Head 98 Tail [ 62, 99, 14 ] (values) [ 98, 62, 99, 14 ] (values) [ 48, 98, 62, 99, 

In [73]:
fixedUpdates |> List.map middle |> List.sum

4713